In [142]:
import pandas as pd

train_features = pd.read_csv('training_set_features.csv')
train_labels = pd.read_csv('training_set_labels.csv')

In [143]:
train_features.count()

respondent_id                  26707
xyz_concern                    26615
xyz_knowledge                  26591
behavioral_antiviral_meds      26636
behavioral_avoidance           26499
behavioral_face_mask           26688
behavioral_wash_hands          26665
behavioral_large_gatherings    26620
behavioral_outside_home        26625
behavioral_touch_face          26579
doctor_recc_xyz                24547
doctor_recc_seasonal           24547
chronic_med_condition          25736
child_under_6_months           25887
health_worker                  25903
health_insurance               14433
opinion_xyz_vacc_effective     26316
opinion_xyz_risk               26319
opinion_xyz_sick_from_vacc     26312
opinion_seas_vacc_effective    26245
opinion_seas_risk              26193
opinion_seas_sick_from_vacc    26170
age_group                      26707
education                      25300
race                           26707
sex                            26707
income_poverty                 22284
m

In [144]:
train_labels.head()


,respondent_id,xyz_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0


In [145]:
train_features


,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,qufhixun,Non-MSA,0.0,0.0,NaN,NaN
26703,26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Rent,Employed,lzgpxyit,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea
26704,26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,NaN,Not Married,Own,NaN,lzgpxyit,"MSA, Not Principle City",0.0,0.0,NaN,NaN
26705,26705,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"<= $75,000, Above Poverty",Married,Rent,Employed,lrircsnp,Non-MSA,1.0,0.0,fcxhlnwr,haliazsg


In [146]:
# i have not dropped the NaN rows cause it would decrease the data set ----> then we should decrese the train_label also
#cause the total values in the train_features and train_labels are same in number i.e 26707


In [147]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

X = train_features.drop(columns=['respondent_id',])
y_xyz = train_labels['xyz_vaccine']
y_seasonal = train_labels['seasonal_vaccine']

numerical_cols = X.select_dtypes(include=['float64','int64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()



In [148]:
numerical_transformer = SimpleImputer(strategy='mean')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])



In [149]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


In [150]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train_xyz, y_val_xyz, y_train_seasonal, y_val_seasonal = train_test_split(
    X, y_xyz, y_seasonal, test_size=0.2, random_state=42)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000,penalty='l2', C=0.1)
'''from sklearn.naive_bayes import GaussianNB
model=GaussianNB()'''             '''--------------------------->not satisfactory'''
'''from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()'''          '''--------------------------->not satisfactory'''


pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

pipeline.fit(X_train, y_train_xyz)
y_pred_xyz = pipeline.predict_proba(X_val)[:, 1]


pipeline.fit(X_train, y_train_seasonal)
y_pred_seasonal = pipeline.predict_proba(X_val)[:, 1]


In [151]:


from sklearn.metrics import roc_auc_score
roc_auc_xyz = roc_auc_score(y_val_xyz, y_pred_xyz)
roc_auc_seasonal = roc_auc_score(y_val_seasonal, y_pred_seasonal)

roc_auc_xyz, roc_auc_seasonal

(0.834385321332224, 0.8564276279978622)

In [152]:
test_features = pd.read_csv('test_set_features.csv')
respondent_ids = test_features['respondent_id']
X_test = test_features.drop(columns=['respondent_id'])


In [153]:
# Fit model on full training set
pipeline.fit(X, y_xyz)
y_test_pred_xyz = pipeline.predict_proba(X_test)[:, 1]

pipeline.fit(X, y_seasonal)
y_test_pred_seasonal = pipeline.predict_proba(X_test)[:, 1]



In [154]:
submission_df = pd.DataFrame({
    'respondent_id': respondent_ids,
    'xyz_vaccine': y_test_pred_xyz,
    'seasonal_vaccine': y_test_pred_seasonal
})


In [156]:
submission_path = 'C:\\Users\\Chethan\\Desktop\\JUPYTER\\Submissionfinal.csv'
submission_df.to_csv(submission_path, index=False)

submission_path


'C:\\Users\\Chethan\\Desktop\\JUPYTER\\Submissionfinal.csv'